In [ ]:
!pip -q install "qiskit>=1.0" "qiskit-aer>=0.14" pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, json, math, random, time
import numpy as np
import pandas as pd

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, SparsePauliOp

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.noise.errors import depolarizing_error, pauli_error, amplitude_damping_error

from qiskit import qpy
import io
import base64

# =========================
# 재현성(시드 고정)
# =========================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# =========================
# 저장 경로
# =========================
OUT_DIR = "/content/drive/MyDrive/qem_dataset"
os.makedirs(OUT_DIR, exist_ok=True)

OUT_CSV = os.path.join(OUT_DIR, "dataset_raw.csv")
OUT_META = os.path.join(OUT_DIR, "run_meta.json")

print("OUT_DIR:", OUT_DIR)


OUT_DIR: /content/drive/MyDrive/qem_dataset


In [ ]:
GATE_POOL_1Q = ["rx", "ry", "rz", "x", "y", "z"]
GATE_POOL_2Q = ["cx"]

def random_2q_circuit(depth: int, p_cx: float = 0.25) -> QuantumCircuit:
    """
    depth 만큼 랜덤하게 게이트를 쌓는 2-qubit 회로 생성
    - p_cx 확률로 CX, 나머지는 1-qubit gate
    - Rx/Ry/Rz는 angle ~ Uniform(0, 2π)
    """
    qc = QuantumCircuit(2)

    for _ in range(depth):
        use_cx = (random.random() < p_cx)
        if use_cx:
            # control/target 랜덤
            if random.random() < 0.5:
                qc.cx(0, 1)
            else:
                qc.cx(1, 0)
        else:
            g = random.choice(GATE_POOL_1Q)
            q = random.choice([0, 1])

            if g in ["rx", "ry", "rz"]:
                theta = random.uniform(0, 2 * math.pi)
                getattr(qc, g)(theta, q)
            else:
                getattr(qc, g)(q)

    return qc

In [ ]:
NOISE_CLASSES = ["noiseless", "pauli", "depol", "amp_damp"]

def sample_noise_params(noise_class: str):
    """
    노이즈 강도 샘플링 (원하는 범위로 조절 가능)
    - depol: p in [0.001, 0.02]
    - pauli: p in [0.001, 0.02] (X/Y/Z 균등)
    - amp_damp: gamma in [0.001, 0.05]
    """
    if noise_class == "depol":
        p = random.uniform(0.001, 0.02)
        return {"p": p}
    if noise_class == "pauli":
        p = random.uniform(0.001, 0.02)
        # X/Y/Z 균등, I는 1-p
        return {"px": p/3, "py": p/3, "pz": p/3}
    if noise_class == "amp_damp":
        gamma = random.uniform(0.001, 0.05)
        return {"gamma": gamma}
    return {}  # noiseless

def build_noise_model(noise_class: str, params: dict) -> NoiseModel:
    """
    Qiskit Aer NoiseModel 생성.
    간단/재현성 좋게: 주로 1-qubit 게이트에 노이즈를 붙이고,
    CX에는 depol일 때만 2-qubit depolarizing을 약하게 추가(옵션).
    """
    nm = NoiseModel()

    oneq_gates = ["rx", "ry", "rz", "x", "y", "z"]
    twoq_gates = ["cx"]

    if noise_class == "noiseless":
        return nm

    if noise_class == "depol":
        p = params["p"]
        err1 = depolarizing_error(p, 1)
        # 2-qubit은 보통 더 약/강하게 설정 가능. 여기선 예시로 p2=2p (원하면 바꿔)
        err2 = depolarizing_error(min(1.0, 2*p), 2)

        for g in oneq_gates:
            nm.add_all_qubit_quantum_error(err1, g)
        nm.add_all_qubit_quantum_error(err2, "cx")
        return nm

    if noise_class == "pauli":
        px, py, pz = params["px"], params["py"], params["pz"]
        pi = max(0.0, 1.0 - (px + py + pz))
        err1 = pauli_error([("X", px), ("Y", py), ("Z", pz), ("I", pi)])

        for g in oneq_gates:
            nm.add_all_qubit_quantum_error(err1, g)
        return nm

    if noise_class == "amp_damp":
        gamma = params["gamma"]
        err1 = amplitude_damping_error(gamma)

        for g in oneq_gates:
            nm.add_all_qubit_quantum_error(err1, g)
        return nm

    raise ValueError("Unknown noise_class")

In [ ]:
ZZ = SparsePauliOp.from_list([("ZZ", 1.0)])

def compute_y_true_expectation(qc_no_measure: QuantumCircuit) -> float:
    """
    noiseless 기대값 y_true = <ZZ>
    - 측정이 없는 회로를 statevector로 시뮬레이션 후 expectation 계산
    """
    sv = Statevector.from_instruction(qc_no_measure)
    val = np.real(sv.expectation_value(ZZ))
    return float(val)

In [ ]:
def run_noisy_counts(qc_with_measure: QuantumCircuit, noise_model: NoiseModel, shots: int = 4096) -> dict:
    """
    AerSimulator로 noisy shots 실행 후 counts(dict) 반환
    """
    sim = AerSimulator(noise_model=noise_model)
    job = sim.run(qc_with_measure, shots=shots)
    result = job.result()
    counts = result.get_counts(0)
    # counts 예: {'00': 1024, '01': 980, ...}
    return counts

In [ ]:
def zz_from_counts(counts: dict) -> float:
    shots = sum(counts.values())
    return (
        counts.get("00", 0)
      - counts.get("01", 0)
      - counts.get("10", 0)
      + counts.get("11", 0)
    ) / shots

In [ ]:
def make_dataset(
    N: int = 5600,
    shots: int = 4096,
    save_every: int = 200,
    p_cx: float = 0.25,
):
    """
    (1) 회로 생성 -> (2) 노이즈 샘플링 -> (3) y_true 계산 -> (4) noisy counts 생성
    결과는 OUT_CSV에 append 저장.
    """

    # 이미 저장된 CSV가 있으면 이어서 생성(런타임 끊겨도 이어가기)
    if os.path.exists(OUT_CSV):
        prev = pd.read_csv(OUT_CSV)
        start_idx = int(prev["circuit_id"].max()) + 1 if len(prev) > 0 else 0
        print(f"[Resume] Existing rows={len(prev)}, start_idx={start_idx}")
    else:
        start_idx = 0
        print("[New] Start from 0")

    rows_buffer = []
    t0 = time.time()

    for cid in range(start_idx, N):
        depth = random.randint(1, 30)

        # (1) circuit 만들기 (측정 없는 버전)
        qc = random_2q_circuit(depth=depth, p_cx=p_cx)

        # (3) y_true (측정 없는 상태에서 계산)
        y_true = compute_y_true_expectation(qc)

        # noisy 실행용 측정 회로 복사 후 measure_all 추가
        qc_m = qc.copy()
        qc_m.measure_all()

        # (2) 노이즈 클래스/강도 샘플링
        noise_class = random.choice(NOISE_CLASSES)
        params = sample_noise_params(noise_class)
        nm = build_noise_model(noise_class, params)

        # (4) noisy counts 얻기
        counts = run_noisy_counts(qc_m, nm, shots=shots)

        zz_noisy = zz_from_counts(counts)

        # QuantumCircuit -> QPY(binary) -> base64(str)
        buf = io.BytesIO()
        qpy.dump(qc, buf)
        qpy_b64 = base64.b64encode(buf.getvalue()).decode("utf-8")


        # 저장 레코드(최소 필드)
        rows_buffer.append({
            "circuit_id": cid,
            "depth": depth,
            "noise_class": noise_class,
            "noise_params": json.dumps(params, ensure_ascii=False),
            "shots": shots,
            "y_true_zz": y_true,
            "zz_noisy": zz_noisy,
            "qpy": qpy_b64,
            "counts": json.dumps(counts),   # dict를 문자열로 저장
        })

        # 중간 저장
        if (cid + 1) % save_every == 0 or (cid + 1) == N:
            df = pd.DataFrame(rows_buffer)
            header = not os.path.exists(OUT_CSV)
            df.to_csv(OUT_CSV, mode="a", header=header, index=False)

            rows_buffer = []
            elapsed = time.time() - t0
            print(f"[Saved] up to circuit_id={cid} | elapsed={elapsed:.1f}s | file={OUT_CSV}")

    # 메타 저장
    meta = {
        "N": N,
        "shots": shots,
        "save_every": save_every,
        "p_cx": p_cx,
        "seed": SEED,
        "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "out_csv": OUT_CSV,
    }
    with open(OUT_META, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print("Done. Meta saved:", OUT_META)

In [ ]:
# 본 실험 (논문 예시급)
make_dataset(N=5600, shots=4096, save_every=200)

[New] Start from 0
[Saved] up to circuit_id=199 | elapsed=3.0s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=399 | elapsed=7.6s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=599 | elapsed=11.2s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=799 | elapsed=14.2s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=999 | elapsed=17.1s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=1199 | elapsed=21.9s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=1399 | elapsed=25.5s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=1599 | elapsed=28.4s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=1799 | elapsed=31.4s | file=/content/drive/MyDrive/qem_dataset/dataset_raw.csv
[Saved] up to circuit_id=1999 | elapsed=36.2s | file

In [80]:
# =========================================================
# Regression-only ML-QEM (paper-style features) + baselines
# - No classification
# - Feature: measurement (per-bitstring) + mean/var + gate features + depth
# - Compare vs baseline(no mitigation) and optional baseline regression
# =========================================================

import os, json, base64, io
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from qiskit import qpy

# -------------------------
# 0) Reproducibility
# -------------------------
def set_seed(seed: int = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# -------------------------
# 1) counts -> feature
#    - per-bitstring feature (paper: "counts transformed to separate features")
#    - mean / variance of measurement outcomes
# -------------------------
BITSTR_2Q = ["00", "01", "10", "11"]

def counts_to_prob_vec(counts: dict, bit_list=BITSTR_2Q) -> np.ndarray:
    """counts dict -> probability vector in fixed bitstring order."""
    shots = max(1, sum(counts.values()))
    return np.array([counts.get(b, 0) / shots for b in bit_list], dtype=np.float32)

def mean_var_from_counts_intmap(counts: dict) -> tuple[float, float]:
    """
    Treat bitstring as integer outcome (e.g., '10'->2), compute mean/variance.
    Matches paper's mean/variance idea from measurement outcomes.
    """
    shots = max(1, sum(counts.values()))
    mu = 0.0
    e2 = 0.0
    for b, c in counts.items():
        v = int(b, 2)
        p = c / shots
        mu += v * p
        e2 += (v**2) * p
    var = e2 - mu**2
    return float(mu), float(var)

# -------------------------
# 2) qpy -> gate features
#    paper: gate sequence -> each gate type as separate feature
# -------------------------
GATE_LIST = ["rx", "ry", "rz", "x", "y", "z", "cx"]

def qpy_b64_to_circuit(qpy_b64: str):
    raw = base64.b64decode(qpy_b64.encode("utf-8"))
    buf = io.BytesIO(raw)
    circuits = qpy.load(buf)
    return circuits[0]

def gate_features(qc, gate_list=GATE_LIST, mode="count") -> np.ndarray:
    """
    mode:
      - 'count': count occurrences of each gate type
      - 'binary': 0/1 whether gate appears
    """
    counts = {g: 0 for g in gate_list}
    for inst, qargs, cargs in qc.data:
        name = inst.name
        if name in counts:
            counts[name] += 1

    vec = np.array([counts[g] for g in gate_list], dtype=np.float32)
    if mode == "binary":
        vec = (vec > 0).astype(np.float32)
    return vec

# -------------------------
# 3) Build feature matrix X, targets
# -------------------------
def build_dataset(csv_path: str,
                  require_2q=True,
                  gate_mode="count",
                  use_prob=True):
    """
    Reads dataset_raw.csv and builds:
      X: [meas_feature(4) + mean + var + gate_feature(len(GATE_LIST)) + depth]
      y_true: noiseless expectation value (target)
      y_noisy: noisy expectation value (baseline)
    """
    df = pd.read_csv(csv_path)

    X_list, y_true_list, y_noisy_list = [], [], []

    for i, row in df.iterrows():
        counts = json.loads(row["counts"])
        # Optional safety check: ensure 2-qubit bitstrings
        if require_2q:
            k = next(iter(counts.keys()))
            if len(k) != 2:
                raise ValueError(f"2큐비트 row가 섞였습니다 (row={i}, key='{k}')")

        meas_vec = counts_to_prob_vec(counts) if use_prob else \
                   np.array([counts.get(b, 0) for b in BITSTR_2Q], dtype=np.float32)

        mu, var = mean_var_from_counts_intmap(counts)

        qc = qpy_b64_to_circuit(row["qpy"])
        g_vec = gate_features(qc, mode=gate_mode)

        depth = float(row["depth"])

        feat = np.concatenate([
            meas_vec,
            np.array([mu, var], dtype=np.float32),
            g_vec,
            np.array([depth], dtype=np.float32),
        ])

        X_list.append(feat)
        y_true_list.append(float(row["y_true_zz"]))
        y_noisy_list.append(float(row["zz_noisy"]))

    X = np.vstack(X_list).astype(np.float32)
    y_true = np.array(y_true_list, dtype=np.float32)
    y_noisy = np.array(y_noisy_list, dtype=np.float32)
    return X, y_true, y_noisy

# -------------------------
# 4) PyTorch regressor (MLP)
# -------------------------
class NPDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float().view(-1, 1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

class RegressorMLP(nn.Module):
    def __init__(self, in_dim: int, hidden=(128, 64), dropout=0.1):
        super().__init__()
        layers = []
        prev = in_dim
        for h in hidden:
            layers += [nn.Linear(prev, h), nn.ReLU(), nn.Dropout(dropout)]
            prev = h
        layers += [nn.Linear(prev, 1)]
        self.net = nn.Sequential(*layers)

    def forward(self, x): return self.net(x)

def train_regressor(model, X_tr, y_tr, X_val, y_val,
                    lr=1e-3, batch=64, epochs=400,
                    patience=40, device="cpu"):
    """
    Train MLP regressor with early stopping on val MSE.
    """
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    tr_loader = DataLoader(NPDataset(X_tr, y_tr), batch_size=batch, shuffle=True)
    val_loader = DataLoader(NPDataset(X_val, y_val), batch_size=batch, shuffle=False)

    best_val = float("inf")
    best_state = None
    bad = 0

    for ep in range(1, epochs + 1):
        model.train()
        for xb, yb in tr_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = loss_fn(pred, yb)
            opt.zero_grad()
            loss.backward()
            opt.step()

        model.eval()
        with torch.no_grad():
            vals = []
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                vals.append(loss_fn(model(xb), yb).item())
            val_mse = float(np.mean(vals))

        if val_mse < best_val - 1e-10:
            best_val = val_mse
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad = 0
        else:
            bad += 1

        if ep % 50 == 0:
            print(f"[Reg] ep={ep:4d}  val_mse={val_mse:.6f}  best={best_val:.6f}  bad={bad}/{patience}")

        if bad >= patience:
            print(f"[Reg] Early stop at ep={ep}, best_val={best_val:.6f}")
            break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

@torch.no_grad()
def predict_regressor(model, X, device="cpu") -> np.ndarray:
    model.eval()
    y = model(torch.from_numpy(X).to(device)).cpu().numpy().reshape(-1)
    return y

# -------------------------
# 5) Evaluation helpers
# -------------------------
def eval_metrics(y_true, y_pred, name=""):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"\n[{name}] MSE={mse:.6f}  MAE={mae:.6f}")
    return mse, mae

def mitigation_stats(y_true, y_noisy, y_miti, name="Mitigation"):
    err_noisy = np.abs(y_true - y_noisy)
    err_miti  = np.abs(y_true - y_miti)
    print(f"\n[{name}] mean|true-noisy|     = {float(np.mean(err_noisy)):.6f}")
    print(f"[{name}] mean|true-mitigated| = {float(np.mean(err_miti)):.6f}")
    print(f"[{name}] improved fraction    = {float(np.mean(err_miti < err_noisy)):.6f}")

# -------------------------
# 6) Main: regression-only pipeline + baselines
# -------------------------
def run_regression_only_experiment(
    csv_path: str,
    test_size=0.2,
    seed=42,
    scaler_type="minmax",   # 'minmax' (paper-style normalization) or 'standard'
    gate_mode="count",      # 'count' or 'binary'
    device=None,
):
    """
    Runs:
      (A) Baseline-0: No mitigation -> predict = zz_noisy
      (B) Baseline-1: Simple regression with ONLY measurement prob features (+mean/var) (optional reference)
      (C) Ours: Regression with full paper-style features (meas + mean/var + gate + depth)
    """
    set_seed(seed)
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # ---- Full feature dataset (paper-style)
    X_full, y_true, y_noisy = build_dataset(
        csv_path,
        require_2q=True,
        gate_mode=gate_mode,
        use_prob=True
    )
    print(f"[Loaded] X_full={X_full.shape}  y_true={y_true.shape}")

    # Train/test split (same indices for all comparisons)
    idx = np.arange(len(y_true))
    tr_idx, te_idx = train_test_split(idx, test_size=test_size, random_state=seed)

    Xtr_full, Xte_full = X_full[tr_idx], X_full[te_idx]
    ytr, yte = y_true[tr_idx], y_true[te_idx]
    ynoisy_te = y_noisy[te_idx]

    # ---- Baseline-0: no mitigation
    print("\n========== Baseline-0: No mitigation (predict = zz_noisy) ==========")
    eval_metrics(yte, ynoisy_te, name="NoMitigation")
    # mitigation_stats 의미는 '모델'이 없으니 생략

    # ---- Scaling for NN stability
    if scaler_type == "minmax":
        scaler_full = MinMaxScaler()
    else:
        scaler_full = StandardScaler()

    Xtr_full_s = scaler_full.fit_transform(Xtr_full).astype(np.float32)
    Xte_full_s = scaler_full.transform(Xte_full).astype(np.float32)

    # internal train/val split
    Xtr_s, Xval_s, ytr2, yval2 = train_test_split(
        Xtr_full_s, ytr, test_size=0.2, random_state=seed
    )

    # ---- (C) Ours: full-feature regressor
    print("\n========== Ours: Regression (full features) ==========")
    model_full = RegressorMLP(in_dim=Xtr_full_s.shape[1], hidden=(128, 64), dropout=0.1).to(device)
    model_full = train_regressor(
        model_full, Xtr_s, ytr2, Xval_s, yval2,
        lr=1e-3, batch=64, epochs=400, patience=40,
        device=device
    )

    ypred_full = predict_regressor(model_full, Xte_full_s, device=device)
    eval_metrics(yte, ypred_full, name="Ours(full)")
    mitigation_stats(yte, ynoisy_te, ypred_full, name="Ours(full) vs Noisy")

    # ---- (B) Baseline-1: simpler regression (measurement probs + mean/var only)
    # feature slice: meas(4) + mean + var = first 6 dims
    # (논문 feature 중 gate/depth 제외한 약한 베이스라인으로 비교)
    print("\n========== Baseline-1: Regression (meas probs + mean/var only) ==========")
    X_meas = X_full[:, :6]
    Xtr_m, Xte_m = X_meas[tr_idx], X_meas[te_idx]

    if scaler_type == "minmax":
        scaler_m = MinMaxScaler()
    else:
        scaler_m = StandardScaler()

    Xtr_m_s = scaler_m.fit_transform(Xtr_m).astype(np.float32)
    Xte_m_s = scaler_m.transform(Xte_m).astype(np.float32)

    Xtrm_s, Xvalm_s, ytrm, yvalm = train_test_split(
        Xtr_m_s, ytr, test_size=0.2, random_state=seed
    )

    model_meas = RegressorMLP(in_dim=Xtr_m_s.shape[1], hidden=(64, 32), dropout=0.1).to(device)
    model_meas = train_regressor(
        model_meas, Xtrm_s, ytrm, Xvalm_s, yvalm,
        lr=1e-3, batch=64, epochs=400, patience=40,
        device=device
    )

    ypred_meas = predict_regressor(model_meas, Xte_m_s, device=device)
    eval_metrics(yte, ypred_meas, name="Baseline-1(meas)")
    mitigation_stats(yte, ynoisy_te, ypred_meas, name="Baseline-1(meas) vs Noisy")

    # ---- Summary comparison (MAE 기준으로 한 줄 비교)
    mae_noisy = mean_absolute_error(yte, ynoisy_te)
    mae_meas  = mean_absolute_error(yte, ypred_meas)
    mae_full  = mean_absolute_error(yte, ypred_full)

    print("\n========== Summary (MAE lower is better) ==========")
    print(f"NoMitigation(zz_noisy) : {mae_noisy:.6f}")
    print(f"Baseline-1(meas only)  : {mae_meas:.6f}")
    print(f"Ours(full features)    : {mae_full:.6f}")

    return {
        "model_full": model_full,
        "scaler_full": scaler_full,
        "model_meas": model_meas,
        "scaler_meas": scaler_m,
        "test_indices": te_idx,
    }

# =========================================================
# Run (Colab)
# =========================================================
results = run_regression_only_experiment(
    "/content/drive/MyDrive/qem_dataset/dataset_raw.csv",
    test_size=0.2,
    seed=42,
    scaler_type="minmax",  # 논문처럼 0~1 정규화 느낌을 원하면 minmax 추천
    gate_mode="count",     # gate를 'binary'로 바꾸면 "게이트 존재 여부" 버전
)


/tmp/ipython-input-1388914033.py:77: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, cargs in qc.data:


[Loaded] X_full=(5600, 14)  y_true=(5600,)

========== Baseline-0: No mitigation (predict = zz_noisy) ==========

[NoMitigation] MSE=0.007062  MAE=0.052443

========== Ours: Regression (full features) ==========
[Reg] ep=  50  val_mse=0.004163  best=0.003967  bad=6/40
[Reg] ep= 100  val_mse=0.004080  best=0.003920  bad=14/40
[Reg] Early stop at ep=126, best_val=0.003920

[Ours(full)] MSE=0.003787  MAE=0.043391

[Ours(full) vs Noisy] mean|true-noisy|     = 0.052443
[Ours(full) vs Noisy] mean|true-mitigated| = 0.043391
[Ours(full) vs Noisy] improved fraction    = 0.540179

========== Baseline-1: Regression (meas probs + mean/var only) ==========
[Reg] ep=  50  val_mse=0.004032  best=0.003970  bad=4/40
[Reg] ep= 100  val_mse=0.004108  best=0.003873  bad=11/40
[Reg] ep= 150  val_mse=0.004107  best=0.003837  bad=33/40
[Reg] Early stop at ep=157, best_val=0.003837

[Baseline-1(meas)] MSE=0.003704  MAE=0.041703

[Baseline-1(meas) vs Noisy] mean|true-noisy|     = 0.052443
[Baseline-1(meas) vs 